# Recipe for Downloading XML Data From food.gov.uk

This notebook is a sketchpad for creating a downloader and database assembler using data obtained from XML files published by the [UK Food Standards Agency](http://ratings.food.gov.uk/open-data/en-GB).

The intention is to represent data downloaded as XML in a tabular *pandas* dataframe, and then persist it using a simple SQLite3 database.

In [1]:
#!pip3 install pandas
#!pip3 install xmltodict

In [3]:
#Test files
url='http://ratings.food.gov.uk/OpenDataFiles/FHRS867en-GB.xml'
url='http://ratings.food.gov.uk/OpenDataFiles/FHRS297en-GB.xml'
url='http://ratings.food.gov.uk/OpenDataFiles/FHRS109en-GB.xml'

## Download a data file

In [4]:
import requests
r=requests.get(url)

We can convert the XML to a *python* `dict`.

In [5]:
import xmltodict
dd=xmltodict.parse(r.text)

In [6]:
##View the output (may be large)
#dd

The XML structure is quite flat.

```
OrderedDict([('FHRSEstablishment',
              OrderedDict([('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('Header',
                            OrderedDict([('ExtractDate', '2018-01-13'),
                                         ('ItemCount', '1146'),
                                         ('ReturnCode', 'Success')])),
                           ('EstablishmentCollection',
                            OrderedDict([('EstablishmentDetail',
                                          [OrderedDict([('FHRSID', '854742'),
                                                        ('LocalAuthorityBusinessID',
                                                         '99301'),
                                                        ('BusinessName',
                                                         '1st Stop Cafe'),
                                                        ('BusinessType',
                                                         'Restaurant/Cafe/Canteen'),
                                                        ...
                                                        ('Scores',
                                                         OrderedDict([('Hygiene',
                                                                       '5'),
                                                                      ('Structural',
                                                                       '5'),
                                                                      ('ConfidenceInManagement',
                                                                       '10')])),
```

The *pandas* package has a package (`from pandas.io.json import json_normalize`) that can normalise JSON data and insert it into a *pandas* dataframe. However, in testing across various files, the automated normalisation does not seem to guarantee consistent output dataframe structures. 

We have more control over the construction of a dataframe if we handle transformations of nested structure ourselves and create a simple dataframe from the `EstablishmentDetail` list of records.

In [7]:
import pandas as pd

In [8]:
dj=pd.DataFrame(dd['FHRSEstablishment']['EstablishmentCollection']['EstablishmentDetail'])
dj['RatingDate']=pd.to_datetime(dj['RatingDate'], errors='coerce')

In [9]:
#The Scores and Geocode columns have nested structure which we can flatten
dj = pd.concat([dj.drop(['Scores'], axis=1), dj['Scores'].apply(pd.Series)], axis=1)
dj = pd.concat([dj.drop(['Geocode'], axis=1), dj['Geocode'].apply(pd.Series)], axis=1)
pass

In [10]:
dj.tail()

,FHRSID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,...,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,SchemeType,NewRatingPending,ConfidenceInManagement,Hygiene,Structural,Latitude,Longitude
1141,89106,23690,Yummy's,Mobile caterer,7846,Startrite Motors,Wrexham Road,Basildon,Essex,SS15 6PX,...,Basildon,http://www.basildon.gov.uk,ehs@basildon.gov.uk,FHRS,False,10,0,5,51.56858100000000,0.42052200000000
1142,738543,38900,Zack's,Mobile caterer,7846,NaN,NaN,NaN,NaN,NaN,...,Basildon,http://www.basildon.gov.uk,ehs@basildon.gov.uk,FHRS,False,5,5,0,NaN,NaN
1143,69076,15496,Zaffran,Takeaway/sandwich shop,7844,NaN,10-14 The Arcade,The Broadway,"Wickford,Essex",SS11 7AH,...,Basildon,http://www.basildon.gov.uk,ehs@basildon.gov.uk,FHRS,False,5,5,5,51.61412700000000,0.52153500000000
1144,830747,43088,Zaika,Takeaway/sandwich shop,7844,The Jolly Friar Public House,Whitmore Way,Basildon,Essex,SS14 3JT,...,Basildon,http://www.basildon.gov.uk,ehs@basildon.gov.uk,FHRS,False,5,5,5,51.58234100000000,0.48670300000000
1145,100896,32692,Zara Indian Takeaway,Takeaway/sandwich shop,7844,NaN,32 High Road,Laindon,Basildon,SS15 6NR,...,Basildon,http://www.basildon.gov.uk,ehs@basildon.gov.uk,FHRS,False,5,10,5,51.56963200000000,0.42187700000000


## Scraping the Data Files

Links to the data files are listed on the page http://ratings.food.gov.uk/open-data/en-GB .

We can write a simple scraper using `BeautifulSoup` to scrape the links from the Food Standards Agency web page.

On the current page, the *English Language* link text is a useful crib for identifying the links.

In [23]:
from bs4 import BeautifulSoup

def getDataHTML(url='http://ratings.food.gov.uk/open-data/en-GB'):
    html=requests.get(url)
    return html

def _getDataList(html):
    def span(cell):
        return cell.find('span').text
    
    soup=BeautifulSoup(html.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    souptables=soup.find("div",{'id':'openDataStatic'}).findAll('tbody')
    items=[]
    th=soup.find("div",{'id':'openDataStatic'}).find('thead').findAll('th')
    header = [span(th[i]) for i in range(len(th))]
    for table in souptables:
        for tr in table.findAll('tr'):
            td = tr.find_all("td")
            a=td[3].find('a')
            if a.text=='English language':
                items.append( (span(td[0]),span(td[1]), span(td[2]),a['href'] ) )
    df=pd.DataFrame(items)
    df.columns=header
    return df

def getDataList():
    #Get the download page HTML
    html = getDataHTML()
    #Extract the links
    df = _getDataList(html)
    return df


In [24]:
df = getDataList()
df.head()

,Local authority,Last update,Number of businesses,Download
0,Babergh,16/01/2018,881,http://ratings.food.gov.uk/OpenDataFiles/FHRS2...
1,Basildon,13/01/2018,"1,146",http://ratings.food.gov.uk/OpenDataFiles/FHRS1...
2,Bedford,28/12/2017,"1,397",http://ratings.food.gov.uk/OpenDataFiles/FHRS7...
3,Braintree,11/01/2018,"1,068",http://ratings.food.gov.uk/OpenDataFiles/FHRS1...
4,Breckland,14/01/2018,"1,256",http://ratings.food.gov.uk/OpenDataFiles/FHRS2...


In [14]:
#How many links are there?
len(df)

392

## Download and Save into a SQLite3 Database

We can now write a routine to download and save data to a simple database.

In testing, it turned out that not all download files returned the dataframes with the same column structure. Some datafiles include *Score* structural elements that were not included in other files. If these richer structure files were download after the SQLite table had been constructed, an error was thrown when trying to add *pandas* generated rows that had columns additional to the columns already in the database table.

A routine was added to detect additional columns in rows and amend the database table prior to trying to add them.

In [ ]:
#Default database filename
dbname='fsafull.db'

In [ ]:
#Delete the database for a clean sweep
#!rm {dbname}

We can save this information, which includes file update dates, as a database table. This could be handy if we extend the recipe to allow for data updates as new files are published.

(The update will require an upsert because new data files are also likely to contain unchanged data.)

Create a database connection.

In [ ]:
import sqlite3

conn = sqlite3.connect(dbname)

In [ ]:
#Table to put metadata in
metadata_table = 'fsa_ratings_metadata'

In [ ]:
#For now, create a new table each time we try to save the metadata to the db
df.to_sql(metadata_table, conn, index=False, if_exists='replace')

### Download and Add Datasets to the Database

In [ ]:
def checkcols(conn,df,table):
    ''' Add new cols to database table if we want to add rows with additional cols '''
    
    #Check to see if we're trying to add rows containing cols not in the db table
    dbcols = pd.read_sql('PRAGMA table_info("{}");'.format(table),conn)['name']
    newcols = list(set(df.columns) - set(dbcols))
    if newcols:
        c = conn.cursor()
        for newcol in newcols:
            q='ALTER TABLE "{}" ADD COLUMN "{}" TEXT;'.format(table,newcol)
            c.execute(q)
    
def append(conn, df, table):
    ''' Append a new set of data to the database table '''
    
    q="SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(table)
    if len(pd.read_sql(q,conn)):
        checkcols(conn,df,table)
    df.to_sql(table, conn, index=False, if_exists='append')

def save_fsa_data(url, conn, table):
    ''' Download XML data file and add the data to the database '''
    
    r=requests.get(url)
    dd=xmltodict.parse(r.text)
    dj=pd.DataFrame(dd['FHRSEstablishment']['EstablishmentCollection']['EstablishmentDetail'])

    dj['RatingDate']=pd.to_datetime(dj['RatingDate'], errors='coerce')
    dj = pd.concat([dj.drop(['Scores'], axis=1), dj['Scores'].apply(pd.Series)], axis=1)
    dj = pd.concat([dj.drop(['Geocode'], axis=1), dj['Geocode'].apply(pd.Series)], axis=1)
    append(conn, dj, table)

In [ ]:
#https://stackoverflow.com/a/39662359/454773
def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False
#use tqdm_notebook in a Jupyter notebook, tqdm in a script
if is_notebook(): from tqdm import tqdm_notebook as tqdm
else: from tqdm import tqdm 

In [25]:
#This would do the trick for the actual download:
#tmp = df['Download'].apply(save_fsa_data,conn=conn, table=table)
#but it's handy to have a progress bar

def download_all(conn, links,  table):
    for url in tqdm(links):
        save_fsa_data(url, conn, table)

In [26]:
table = 'ratings'
download_all(conn, df['Download'], table)

HBox(children=(IntProgress(value=0, max=392), HTML(value='')))

KeyboardInterrupt: 

## Testing

Now we can have some fun testing the database...

In [20]:
#check columns
q='SELECT * FROM ratings LIMIT 1'
df = pd.read_sql(q,conn)
df.columns.tolist()

['FHRSID',
 'LocalAuthorityBusinessID',
 'BusinessName',
 'BusinessType',
 'BusinessTypeID',
 'RatingValue',
 'RatingKey',
 'RatingDate',
 'LocalAuthorityCode',
 'LocalAuthorityName',
 'LocalAuthorityWebSite',
 'LocalAuthorityEmailAddress',
 'SchemeType',
 'NewRatingPending',
 'AddressLine1',
 'AddressLine2',
 'AddressLine3',
 'PostCode',
 'AddressLine4',
 'ConfidenceInManagement',
 'Hygiene',
 'Structural',
 'Latitude',
 'Longitude']

For example, can we find folk catering to large Amazon sites?

In [194]:
q='''
select BusinessName, BusinessType, AddressLine1, AddressLine2,AddressLine3, PostCode 
from ratings where (lower(AddressLine1) like "%amazon%") OR (lower(BusinessName) LIKE "%amazon%")
'''

pd.read_sql(q,conn)



,BusinessName,BusinessType,AddressLine1,AddressLine2,AddressLine3,PostCode
0,Amazon EU SARL,Retailers - other,Unit DC1 Prologis,Boscombe Road,Dunstable,LU5 4FE
1,Amazon UK Services Ltd,Retailers - other,Unit 8 To Unit 8A,Badgers Rise,Brogborough,MK43 0ZA
2,Baxter Storey,Restaurant/Cafe/Canteen,Amazon,Unit DC1 Prologis,Boscombe Road,LU5 4FE
3,Baxter Storey,Restaurant/Cafe/Canteen,Amazon.co.uk,Unit 8,Badgers Rise,MK43 0ZA
4,Baxter Storey at Amazon Co UK Ltd,Other catering premises,Amazon Co Uk Ltd,Boundary Way,Hemel Hempstead,HP2 7LF
5,Amazona Zoo,Restaurant/Cafe/Canteen,Hall Road,Cromer,Norfolk,NR27 9JG
6,Servest at Amazon,Restaurant/Cafe/Canteen,None,1 Flaxley Road,Kingston Park,PE2 9EN
7,Amazon UK Services Ltd,Distributors/Transporters,30 Watchmead,Welwyn Garden City,Hertfordshire,AL7 1LT
8,Baxter Storey @ Amazon Daventry,Restaurant/Cafe/Canteen,Amazon,Royal Oak Way North,Royal Oak,NN11 8QL
9,Amazon UK Services Ltd,Restaurant/Cafe/Canteen,Amazon Distribution Centre,Robson Way,Ellistown,LE67 1GQ


## Further Work

Items for further exploration.

### Updating the database
It may be useful to be able to update the database as new data files are released.

*pandas* does not natively provide a method for upserting data into a `sqlite` database, but others have explored this feature: https://www.ryanbaumann.com/blog/2016/4/30/python-pandas-tosql-only-insert-new-rows

### Normalising the Data

The data relating to *Scores* was originally structure and then flattened. We could pull the *Scores* data out into a separate table, perhaps with inspection date?